# Gabriel Nogueira Souza e Roberto Fernandes Nascimento Júnior

In [41]:
import pandas as pd
from datetime import datetime as dt

In [42]:
!pip install chardet
import chardet

with open('/content/dadosAulav1.csv', 'rb') as f:
    result = chardet.detect(f.read())

print(result)


{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


## Importar os dados

In [43]:
df_aula = pd.read_csv("/content/dadosAulav1.csv", sep=";", parse_dates=["Data de Nascimento"], date_format="%d/%m/%Y", encoding="ISO-8859-1")

In [44]:
df_aula.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID                  98 non-null     int64         
 1   Nome                98 non-null     object        
 2   Idade               89 non-null     float64       
 3   Data de Nascimento  90 non-null     datetime64[ns]
 4   Mancha              97 non-null     object        
 5   Febre               88 non-null     float64       
 6   Pressão             96 non-null     object        
 7   Doença              96 non-null     object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 6.3+ KB


In [45]:
df_aula.head()

,ID,Nome,Idade,Data de Nascimento,Mancha,Febre,Pressão,Doença
0,0,Alice,61.0,1963-05-02,NÃO,36.0,Baixa,B
1,1,Ana,25.0,1999-01-01,Sim,39.0,Boa,A
2,2,Ana,65.0,1960-01-02,NÃO,35.0,Baixa,B
3,3,Anderson,58.0,1966-01-10,NÃO,36.0,Boa,NaN
4,4,André,24.0,2000-01-25,Sim,39.0,Boa,A


## Valores unicos

In [46]:
var_cols = ["Mancha", "Pressão", "Doença"]

In [47]:
for col in var_cols:
    unique_vals = df_aula[col].unique()
    print(f"Column: {col}")
    print(unique_vals)
    print("-" * 40)

Column: Mancha
['NÃO' 'Sim' 'nao' 'SIM' 'N' 'S' nan]
----------------------------------------
Column: Pressão
['Baixa' 'Boa' 'Alta' nan]
----------------------------------------
Column: Doença
['B' 'A' nan 'Saudável' 'C']
----------------------------------------


### A coluna `Mancha` precisa ser padronizada

## Verificar duplicatas

In [48]:
df_aula.duplicated().sum()

np.int64(1)

In [49]:
df_aula.drop_duplicates(inplace=True);

## Verificar nulos

In [50]:
df_aula.isna().sum()

,0
ID,0
Nome,0
Idade,9
Data de Nascimento,8
Mancha,1
Febre,10
Pressão,2
Doença,2


### Algumas entradas tem apenas duas informacoes e nao sao relevantes

In [51]:
df_aula_empty_rows = df_aula[df_aula.isnull().any(axis=1)].isnull().sum(axis=1).sort_values(ascending=False)
df_aula_empty_rows = df_aula_empty_rows[df_aula_empty_rows > 1] # Rows with more than one empty column
df_aula_empty_rows

,0
48,5
19,2
40,2
73,2
62,2


In [52]:
# Drop rows with more than one empty column

df_aula.drop(df_aula_empty_rows.index, axis=0, inplace=True)

In [53]:
# Depois de retirar as linhas vazias tem uma reducao consideravel do numero variaveis vazias

df_aula.isna().sum()

,0
ID,0
Nome,0
Idade,5
Data de Nascimento,3
Mancha,0
Febre,8
Pressão,1
Doença,2


In [54]:
df_any_null = df_aula[df_aula.isnull().any(axis=1)]
df_any_null

,ID,Nome,Idade,Data de Nascimento,Mancha,Febre,Pressão,Doença
3,3,Anderson,58.0,1966-01-10,NÃO,36.0,Boa,NaN
6,6,Bianca,NaN,1972-05-05,NÃO,36.0,Boa,Saudável
7,7,Bruno,27.0,1997-02-02,nao,NaN,Alta,A
10,10,Carlos,62.0,NaT,NÃO,36.0,Boa,Saudável
12,12,Daniel,24.0,2000-04-04,Sim,NaN,Boa,A
18,18,Elisa,NaN,2002-05-05,Sim,40.0,Alta,A
24,24,Gabriela,26.0,1998-07-07,Sim,38.0,NaN,A
27,27,Gustavo,NaN,1980-04-08,SIM,36.0,Alta,C
33,33,Igor,NaN,1964-05-07,NÃO,36.0,Alta,Saudável
34,34,Ingrid,24.0,2000-09-09,Sim,NaN,Alta,A


### Tratamento de `Idade` nula

In [55]:
df_null_age = df_any_null[df_any_null["Idade"].isna()]
df_null_age

,ID,Nome,Idade,Data de Nascimento,Mancha,Febre,Pressão,Doença
6,6,Bianca,NaN,1972-05-05,NÃO,36.0,Boa,Saudável
18,18,Elisa,NaN,2002-05-05,Sim,40.0,Alta,A
27,27,Gustavo,NaN,1980-04-08,SIM,36.0,Alta,C
33,33,Igor,NaN,1964-05-07,NÃO,36.0,Alta,Saudável
70,70,Sofia,NaN,1954-02-20,NÃO,36.0,Boa,Saudável


In [56]:
df_null_age.loc[~df_null_age["Data de Nascimento"].isna(), "Idade"] = dt.now().year - df_null_age.loc[~df_null_age["Data de Nascimento"].isna(), "Data de Nascimento"].dt.year

median_age = int(df_aula["Idade"].median())
df_null_age.loc[df_null_age["Data de Nascimento"].isna(), "Idade"] = median_age

In [57]:
df_aula.loc[df_null_age.index] = df_null_age

### Agora que a coluna `Data de Nascimento` ja foi utilizada para completar as entradas faltantes em idade, ela pode ser descartada por conter informacoes redundantes

In [58]:
df_aula.drop("Data de Nascimento", axis=1, inplace=True)
df_any_null.drop("Data de Nascimento", axis=1, inplace=True)

<ipython-input-58-1209397895ae>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_any_null.drop("Data de Nascimento", axis=1, inplace=True)


### Tratamento de `Febre` vazia

In [59]:
df_null_fever = df_any_null[df_any_null["Febre"].isna()]
df_null_fever

,ID,Nome,Idade,Mancha,Febre,Pressão,Doença
7,7,Bruno,27.0,nao,NaN,Alta,A
12,12,Daniel,24.0,Sim,NaN,Boa,A
34,34,Ingrid,24.0,Sim,NaN,Alta,A
58,58,Patricia,62.0,NÃO,NaN,Boa,Saudável
61,61,Paulo,25.0,NÃO,NaN,Boa,A
71,71,Tâmara,64.0,Sim,NaN,Baixa,B
82,82,Wagner,56.0,NÃO,NaN,Baixa,B
89,89,Ximena,35.0,NÃO,NaN,Boa,Saudável


In [60]:
fever_mean_healthy = round(df_aula[df_aula["Doença"] == "Saudável"]["Febre"].apply("mean"), 1)
fever_mean_A = round(df_aula[df_aula["Doença"] == "A"]["Febre"].apply("mean"), 1)
fever_mean_B = round(df_aula[df_aula["Doença"] == "B"]["Febre"].apply("mean"), 1)
fever_mean_C = round(df_aula[df_aula["Doença"] == "C"]["Febre"].apply("mean"), 1)

print(fever_mean_healthy)
print(fever_mean_A)
print(fever_mean_B)
print(fever_mean_C)

35.5
39.1
36.4
36.0


In [61]:
df_null_fever.loc[df_null_fever["Doença"] == "Saudável", "Febre"] = fever_mean_healthy
df_null_fever.loc[df_null_fever["Doença"] == "A", "Febre"] = fever_mean_A
df_null_fever.loc[df_null_fever["Doença"] == "B", "Febre"] = fever_mean_B
df_null_fever.loc[df_null_fever["Doença"] == "C", "Febre"] = fever_mean_C

In [62]:
df_aula.loc[df_null_fever.index] = df_null_fever

### Tratamento de `Pressão` vazia


In [63]:
df_null_p = df_any_null[df_any_null["Pressão"].isna()]
df_null_p

,ID,Nome,Idade,Mancha,Febre,Pressão,Doença
24,24,Gabriela,26.0,Sim,38.0,NaN,A


In [64]:
p_mode = df_aula["Pressão"].mode()
print(p_mode)

0    Boa
Name: Pressão, dtype: object


In [65]:
df_null_p["Pressão"] = p_mode

<ipython-input-65-26be3c0dd014>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_null_p["Pressão"] = p_mode


In [66]:
df_aula.loc[df_null_p.index] = df_null_p

### Tratamento de `Doença` vazia

Column: Doença
['B' 'A' nan 'Saudável' 'C']

#### Em relação aos dados, a informação sobre a "Doença" que estiver vazia pode ser considerada como “Saudável”.

In [67]:
df_null_d = df_any_null[df_any_null["Doença"].isna()]
df_null_d

,ID,Nome,Idade,Mancha,Febre,Pressão,Doença
3,3,Anderson,58.0,NÃO,36.0,Boa,NaN
86,86,Xavier,24.0,NÃO,39.0,Boa,NaN


In [68]:
df_null_d["Doença"] = "Saudável"

<ipython-input-68-e78830593f36>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_null_d["Doença"] = "Saudável"


In [69]:
df_aula.loc[df_null_d.index] = df_null_d

## Drop variaveis que nao seriam uteis para previsao/classificacao

In [70]:
variaveis_inuteis = ["ID", "Nome"]

df_aula.drop(variaveis_inuteis, axis=1, inplace=True)

In [71]:
df_aula.head()

,Idade,Mancha,Febre,Pressão,Doença
0,61.0,NÃO,36.0,Baixa,B
1,25.0,Sim,39.0,Boa,A
2,65.0,NÃO,35.0,Baixa,B
3,58.0,NÃO,36.0,Boa,Saudável
4,24.0,Sim,39.0,Boa,A


## Padronizacao de valores

Column: Mancha
['NÃO' 'Sim' 'nao' 'SIM' 'N' 'S' nan]

In [72]:
df_aula.loc[df_aula["Mancha"] == "Sim", "Mancha"] = "SIM"
df_aula.loc[df_aula["Mancha"] == "S", "Mancha"] = "SIM"
df_aula.loc[df_aula["Mancha"] == "nao", "Mancha"] = "NÃO"
df_aula.loc[df_aula["Mancha"] == "N", "Mancha"] = "NÃO"
df_aula.loc[df_aula["Mancha"] == "Sim", "Mancha"] = "NÃO"

In [73]:
df_aula.head()

,Idade,Mancha,Febre,Pressão,Doença
0,61.0,NÃO,36.0,Baixa,B
1,25.0,SIM,39.0,Boa,A
2,65.0,NÃO,35.0,Baixa,B
3,58.0,NÃO,36.0,Boa,Saudável
4,24.0,SIM,39.0,Boa,A


## Encodificacao de variaveis (dummy, ordinais)

### Para os atributos nominais, será necessário utilizar o método de codificação de variáveis dummy.

In [74]:
atrib_nominais = ["Mancha", "Doença"]

In [75]:
df_aula = pd.get_dummies(df_aula, columns=atrib_nominais)
df_aula.head()

,Idade,Febre,Pressão,Mancha_NÃO,Mancha_SIM,Doença_A,Doença_B,Doença_C,Doença_Saudável
0,61.0,36.0,Baixa,True,False,False,True,False,False
1,25.0,39.0,Boa,False,True,True,False,False,False
2,65.0,35.0,Baixa,True,False,False,True,False,False
3,58.0,36.0,Boa,True,False,False,False,False,True
4,24.0,39.0,Boa,False,True,True,False,False,False


### Os atributos ordinais deverão ser mapeados em valores numéricos que representam cada grandeza.

In [76]:
atrib_ordinais = ["Pressão"]

In [77]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()

df_aula[atrib_ordinais] = encoder.fit_transform(df_aula[atrib_ordinais])

df_aula.head()

,Idade,Febre,Pressão,Mancha_NÃO,Mancha_SIM,Doença_A,Doença_B,Doença_C,Doença_Saudável
0,61.0,36.0,1.0,True,False,False,True,False,False
1,25.0,39.0,2.0,False,True,True,False,False,False
2,65.0,35.0,1.0,True,False,False,True,False,False
3,58.0,36.0,2.0,True,False,False,False,False,True
4,24.0,39.0,2.0,False,True,True,False,False,False


## Os valores deverão ser normalizados em um intervalo de 0 a 1.

In [78]:
num_columns = ["Idade", "Febre", "Pressão"]

In [79]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df_aula[num_columns] = scaler.fit_transform(df_aula[num_columns])

df_aula.head()

,Idade,Febre,Pressão,Mancha_NÃO,Mancha_SIM,Doença_A,Doença_B,Doença_C,Doença_Saudável
0,0.195238,0.50,0.5,True,False,False,True,False,False
1,0.023810,0.65,1.0,False,True,True,False,False,False
2,0.214286,0.45,0.5,True,False,False,True,False,False
3,0.180952,0.50,1.0,True,False,False,False,False,True
4,0.019048,0.65,1.0,False,True,True,False,False,False


In [80]:
df_aula.to_csv("/content/dadosAulav1_processados.csv")